# Making DBS input for SigProfiler

In [24]:
import io
import os
import numpy as np
import pandas as pd
import gzip as gz
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker
from matplotlib_venn import venn2, venn3

sns.set_theme(font="Arial", font_scale=1.15, style='ticks') 
matplotlib.rcParams['figure.dpi'] = 150
plt.rc("axes.spines", top=True, right=True)

def read_vcf(path):
    if path[-3:] == ".gz": 
        with gz.open(path, 'rb') as f:
            lines = [l.decode('utf-8') for l in f if not l.startswith(b'##')]
            return pd.read_csv(
                io.StringIO(''.join(lines)),
                dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
                       'QUAL': str, 'FILTER': str, 'INFO': str},
                       sep='\t'
                       ).rename(columns={'#CHROM': 'CHROM'})
    else:
        with open(path, 'r') as f:
            lines = [l for l in f if not l.startswith('##')]
            return pd.read_csv(
                io.StringIO(''.join(lines)),
                dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
                       'QUAL': str, 'FILTER': str, 'INFO': str},
                       sep='\t'
                       ).rename(columns={'#CHROM': 'CHROM'})

dir="/mmfs1/gscratch/stergachislab/mhsohny/SMaHT/Improving_SomaticVariantCalling_through_DSA/Fiber-seq"
DSA="/mmfs1/gscratch/stergachislab/mhsohny/SMaHT/DSA/DSA_COLO829BL_v3.0.0.fasta"

colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt = read_vcf(f"{dir}/VariantCalls_DeepVariant_1.6.1/Mutational_Spectrum/COLO829T_PassageB_DSA.deepvariant.split.snv.modified.final.vcf.gz")
colota_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt = read_vcf(f"{dir}/VariantCalls_DeepVariant_1.6.1/Mutational_Spectrum/COLO829T_PassageA_DSA.deepvariant.split.snv.modified.final.vcf.gz")

colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt['SNVid'] = colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt[['CHROM', 'POS', 'REF', 'ALT']].astype(str).apply('_'.join, axis=1)
colota_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt['SNVid'] = colota_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt[['CHROM', 'POS', 'REF', 'ALT']].astype(str).apply('_'.join, axis=1)

colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt_set = set(colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt['SNVid'].values)
colota_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt_set = set(colota_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt['SNVid'].values)

tba_FlaggerHap_glfilt_pileupfilt_tba_set = colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt_set.intersection(colota_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt_set)
tb_FlaggerHap_glfilt_pileupfilt_onlytb_set = colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt_set.difference(colota_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt_set)
ta_FlaggerHap_glfilt_pileupfilt_onlyta_set = colota_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt_set.difference(colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt_set)

print(f"The number of Shared SNVs between Passage B and A: {len(tba_FlaggerHap_glfilt_pileupfilt_tba_set):,}")
print(f"The number of B specific SNVs: {len(tb_FlaggerHap_glfilt_pileupfilt_onlytb_set):,}")
print(f"The number of A specific SNVs: {len(ta_FlaggerHap_glfilt_pileupfilt_onlyta_set):,}")

print(f"The number of DBS variants for Passage B: {colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt[colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt.groupby("CHROM")['POS'].diff() == 1].shape[0]:,}")
print(f"The number of DBS variants for Passage B: {colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt[colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt.groupby("CHROM")['POS'].diff().shift(-1) == 1].shape[0]:,}")

The number of Shared SNVs between Passage B and A: 51,678
The number of B specific SNVs: 6,577
The number of A specific SNVs: 3,781
The number of DBS variants for Passage B: 1,175
The number of DBS variants for Passage B: 1,175


In [25]:
colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt[colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt.groupby("CHROM")['POS'].diff() == 1]

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,COLO829T_PassageB_DSA,SNVid
117,haplotype1-0000001,12676857,.,C,T,42.4,PASS,.,GT:GQ:DP:AD:VAF:PL,"0/1:12:117:23,94:0.803419:42,0,11",haplotype1-0000001_12676857_C_T
220,haplotype1-0000001,28039723,.,A,G,63,PASS,.,GT:GQ:DP:AD:VAF:PL,"1/1:56:112:0,112:1:62,57,0",haplotype1-0000001_28039723_A_G
234,haplotype1-0000001,29598399,.,C,T,58.5,PASS,.,GT:GQ:DP:AD:VAF:PL,"1/1:56:59:0,59:1:58,58,0",haplotype1-0000001_29598399_C_T
260,haplotype1-0000001,35267980,.,G,A,72.7,PASS,.,GT:GQ:DP:AD:VAF:PL,"1/1:62:93:0,93:1:72,62,0",haplotype1-0000001_35267980_G_A
275,haplotype1-0000001,39873420,.,C,T,66.9,PASS,.,GT:GQ:DP:AD:VAF:PL,"1/1:61:69:0,69:1:66,61,0",haplotype1-0000001_39873420_C_T
...,...,...,...,...,...,...,...,...,...,...,...
58070,haplotype2-0000079,152890010,.,C,T,61.7,PASS,.,GT:GQ:DP:AD:VAF:PL,"1/1:60:64:0,64:1:61,66,0",haplotype2-0000079_152890010_C_T
58084,haplotype2-0000079,154873266,.,G,A,22.7,PASS,.,GT:GQ:DP:AD:VAF:PL,"0/1:23:43:35,8:0.186047:22,0,46",haplotype2-0000079_154873266_G_A
58125,haplotype2-0000079,161251108,.,C,T,56.4,PASS,.,GT:GQ:DP:AD:VAF:PL,"0/1:47:58:9,49:0.844828:56,0,47",haplotype2-0000079_161251108_C_T
58208,haplotype2-0000079,171322182,.,G,A,63,PASS,.,GT:GQ:DP:AD:VAF:PL,"1/1:61:57:0,57:1:63,64,0",haplotype2-0000079_171322182_G_A


In [26]:
colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt[colotb_snvs_pass_annot_FlaggerHap_glfilt_pileupfilt.groupby("CHROM")['POS'].diff().shift(-1) == 1]

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,COLO829T_PassageB_DSA,SNVid
116,haplotype1-0000001,12676856,.,C,T,42.3,PASS,.,GT:GQ:DP:AD:VAF:PL,"0/1:9:117:23,94:0.803419:41,0,8",haplotype1-0000001_12676856_C_T
219,haplotype1-0000001,28039722,.,G,T,65.2,PASS,.,GT:GQ:DP:AD:VAF:PL,"1/1:50:112:0,112:1:65,50,0",haplotype1-0000001_28039722_G_T
233,haplotype1-0000001,29598398,.,C,T,63,PASS,.,GT:GQ:DP:AD:VAF:PL,"1/1:56:59:0,59:1:63,57,0",haplotype1-0000001_29598398_C_T
259,haplotype1-0000001,35267979,.,G,A,66.5,PASS,.,GT:GQ:DP:AD:VAF:PL,"1/1:56:93:0,93:1:66,56,0",haplotype1-0000001_35267979_G_A
274,haplotype1-0000001,39873419,.,C,T,64,PASS,.,GT:GQ:DP:AD:VAF:PL,"1/1:58:69:0,69:1:64,59,0",haplotype1-0000001_39873419_C_T
...,...,...,...,...,...,...,...,...,...,...,...
58069,haplotype2-0000079,152890009,.,C,T,60.7,PASS,.,GT:GQ:DP:AD:VAF:PL,"1/1:59:64:0,64:1:60,64,0",haplotype2-0000079_152890009_C_T
58083,haplotype2-0000079,154873265,.,G,A,18.5,PASS,.,GT:GQ:DP:AD:VAF:PL,"0/1:19:43:35,8:0.186047:18,0,44",haplotype2-0000079_154873265_G_A
58124,haplotype2-0000079,161251107,.,C,T,53.8,PASS,.,GT:GQ:DP:AD:VAF:PL,"0/1:42:58:9,49:0.844828:53,0,42",haplotype2-0000079_161251107_C_T
58207,haplotype2-0000079,171322181,.,G,A,60.7,PASS,.,GT:GQ:DP:AD:VAF:PL,"1/1:55:57:0,57:1:60,56,0",haplotype2-0000079_171322181_G_A
